In [ ]:
!pip install 'transformers[torch]'

In [2]:
# Load model directly
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.data import Dataset

In [3]:
import torch
import numpy as np
import random
from sklearn.metrics import accuracy_score, f1_score
def set_seeds(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
set_seeds(42)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ltrctelugu/bert_ltrc_telugu")
model = AutoModelForSequenceClassification.from_pretrained("ltrctelugu/bert_ltrc_telugu")
model.cuda()

In [13]:
train_args = TrainingArguments(
        'outputs',
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate = 2e-5,
        per_device_train_batch_size = 250,
        per_device_eval_batch_size = 250,
        num_train_epochs = 10,
        weight_decay = 0.01,
        load_best_model_at_end = True,
        metric_for_best_model = 'f1_macro'
    )

In [14]:
class LTRCDataset(Dataset):
    def __init__(self, text, labels, tokenizer, max_len):
        self.text = text
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        text = str(self.text[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [15]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/data/ltrc/ltrc_tel_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/data/ltrc/ltrc_tel_test.csv')
val_df = pd.read_csv('/content/drive/MyDrive/data/ltrc/ltrc_tel_val.csv')

In [16]:
def prepare_dataset(df):
  return LTRCDataset(text=df.text.to_numpy(), labels=df.label_yn.to_numpy(), tokenizer=tokenizer, max_len=128)

In [17]:
train_ds = prepare_dataset(train_df)
test_ds = prepare_dataset(test_df)
val_ds = prepare_dataset(val_df)

In [18]:
def get_metrics(preds, labels):
    acc = accuracy_score(labels, preds)
    f1_micro = f1_score(labels, preds, average='micro')
    f1_macro = f1_score(labels, preds, average='macro')
    print ('jacc acc:{}, f1 micro score:{} f1 macro score:{}'.format(acc, f1_micro, f1_macro))
    return acc, f1_micro, f1_macro

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc, f1_micro, f1_macro = get_metrics(preds, labels)
    print(f"accuracy: {acc}, f1_macro: {f1_macro}, f1_micro: {f1_micro}")
    #return {'accuracy': acc, "f1_macro": f1_macro, "f1_micro": f1_micro}
    return {'f1_macro':f1_macro, 'accuracy':acc}

In [19]:
trainer = Trainer(
        model=model,
        args=train_args,
        train_dataset = train_ds,
        eval_dataset = val_ds,
        tokenizer = tokenizer,
        compute_metrics = compute_metrics
    )

In [21]:
import gc
gc.collect()
torch.cuda.empty_cache()
trainer.train()

Epoch,Training Loss,Validation Loss,F1 Macro,Accuracy
1,No log,0.049789,0.497782,0.991168
2,No log,0.050092,0.497782,0.991168
3,No log,0.049875,0.497782,0.991168
4,No log,0.049865,0.497782,0.991168
5,No log,0.050079,0.497782,0.991168
6,0.061300,0.050044,0.497782,0.991168
7,0.061300,0.050256,0.497782,0.991168
8,0.061300,0.050822,0.497782,0.991168
9,0.061300,0.050035,0.497782,0.991168
10,0.061300,0.050038,0.497782,0.991168


jacc acc:0.9911680911680911, f1 micro score:0.9911680911680911 f1 macro score:0.4977822292173415
accuracy: 0.9911680911680911, f1_macro: 0.4977822292173415, f1_micro: 0.9911680911680911
jacc acc:0.9911680911680911, f1 micro score:0.9911680911680911 f1 macro score:0.4977822292173415
accuracy: 0.9911680911680911, f1_macro: 0.4977822292173415, f1_micro: 0.9911680911680911
jacc acc:0.9911680911680911, f1 micro score:0.9911680911680911 f1 macro score:0.4977822292173415
accuracy: 0.9911680911680911, f1_macro: 0.4977822292173415, f1_micro: 0.9911680911680911
jacc acc:0.9911680911680911, f1 micro score:0.9911680911680911 f1 macro score:0.4977822292173415
accuracy: 0.9911680911680911, f1_macro: 0.4977822292173415, f1_micro: 0.9911680911680911
jacc acc:0.9911680911680911, f1 micro score:0.9911680911680911 f1 macro score:0.4977822292173415
accuracy: 0.9911680911680911, f1_macro: 0.4977822292173415, f1_micro: 0.9911680911680911
jacc acc:0.9911680911680911, f1 micro score:0.9911680911680911 f1 macr

TrainOutput(global_step=990, training_loss=0.05878128475613064, metrics={'train_runtime': 1204.9024, 'train_samples_per_second': 204.158, 'train_steps_per_second': 0.822, 'total_flos': 1.61806721270016e+16, 'train_loss': 0.05878128475613064, 'epoch': 10.0})

In [22]:
test_metrics = trainer.predict(test_ds)

jacc acc:0.9891938006540595, f1 micro score:0.9891938006540595 f1 macro score:0.4972837741243746
accuracy: 0.9891938006540595, f1_macro: 0.4972837741243746, f1_micro: 0.9891938006540595


In [23]:
test_metrics

PredictionOutput(predictions=array([[ 2.2513857, -2.5532396],
       [ 2.2688046, -2.5743685],
       [ 2.2881477, -2.5997906],
       ...,
       [ 2.361202 , -2.6864018],
       [ 2.3157024, -2.6394453],
       [ 2.446384 , -2.8146782]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.05927256494760513, 'test_f1_macro': 0.4972837741243746, 'test_accuracy': 0.9891938006540595, 'test_runtime': 11.9966, 'test_samples_per_second': 586.249, 'test_steps_per_second': 2.417})